In [1]:
#
# Copyright © 2019 Sunho Kim. All rights reserved.
#

In [2]:
cd ..

/gorani/gorani/backend/dataserver


In [3]:
from pyspark.sql import SparkSession, DataFrame

spark = SparkSession\
    .builder\
    .appName('Recommend Books')\
    .getOrCreate()
sc = spark.sparkContext

In [4]:
# parameters
book_number = 1

In [10]:
from gorani.spark import read_api_all, read_data_all

rate_df = read_api_all(spark, 'rates')\
        .where('kind = "recommended_book"')
rate_df.show()

eb_df = read_data_all(spark, 'experienced_books')
eb_df.show()

rb_df = read_data_all(spark, 'readable_books')
rb_df.show()

rcb_df = read_api_all(spark, 'recommended_books')
rcb_df.show()

cluster_df = read_data_all(spark, 'book_cluster')
cluster_df.show()

+--------------------+-------+---------+----------------+----+--------------------+--------------------+
|                  id|user_id|target_id|            kind|rate|          created_at|          updated_at|
+--------------------+-------+---------+----------------+----+--------------------+--------------------+
|ae142f49-b63e-443...|     10|        2|recommended_book|  -1|2019-05-25 21:13:...|2019-05-25 22:42:...|
|21c98ea1-670a-41a...|     10|        3|recommended_book|  -1|2019-05-26 05:23:...|2019-05-26 06:01:...|
|1110fe5a-26af-4bc...|     10|        5|recommended_book|   1|2019-05-26 06:02:...|2019-05-26 06:02:...|
+--------------------+-------+---------+----------------+----+--------------------+--------------------+

+-------+-------+
|user_id|book_id|
+-------+-------+
|     10|      4|
+-------+-------+

+-------+-------+
|user_id|book_id|
+-------+-------+
|     10|      1|
|     10|      2|
|     10|      3|
|     10|      4|
|     10|      5|
|     10|      6|
|      1|  

In [11]:
from pyspark.sql import functions as F
from pyspark.sql.types import IntegerType

cluster_count_df = rate_df.join(cluster_df, rate_df['target_id'] == cluster_df['id'], 'inner')\
    .select(F.col('target_id').alias('book_id'), 'user_id', 'cluster', 'rate')\
    .groupBy('user_id', 'cluster').agg(F.sum('rate').alias('count'))\
    .select('user_id', 'cluster', F.when(F.col('count') < 0, F.lit(0)).otherwise(F.col('count').cast(IntegerType())).alias('count'))

cluster_count_sum_df = cluster_count_df.groupBy('user_id')\
                    .agg(F.sum('count').alias('sum'))

cluster_count_df = cluster_count_df.join(cluster_count_sum_df.alias('f'),\
                    cluster_count_df['user_id'] == cluster_count_sum_df['user_id'], 'left')\
                    .drop(F.col('f.user_id'))

rcbn_df = rcb_df.groupBy('user_id')\
    .agg(F.count(F.lit(1)).alias('old_rec'))\

cluster_count_df = cluster_count_df.join(rcbn_df.alias('f2'),\
        cluster_count_df['user_id'] == rcbn_df['user_id'], 'left')\
        .drop(F.col('f2.user_id'))\
        .select('user_id', 'cluster', 'count', 'sum', F.when(F.isnull('old_rec'), 0).otherwise(F.col('old_rec')).alias('old_rec'))

cluster_count_df.show()

+-------+-------+-----+---+-------+
|user_id|cluster|count|sum|old_rec|
+-------+-------+-----+---+-------+
|     10|      1|    0|  0|      0|
|     10|      0|    0|  0|      0|
+-------+-------+-----+---+-------+



In [12]:
df = cluster_df.join(rb_df, cluster_df['id'] == rb_df['book_id'], 'inner').drop(F.col('id'))
df.show()
df = df.join(eb_df, (df['book_id'] == eb_df['book_id']) & (df['user_id'] == eb_df['user_id']), 'left_anti')
candidate_df = df.join(rate_df, (df['book_id'] == rate_df['target_id']) & (df['user_id'] == rate_df['user_id']), 'left_anti')
candidate_df.show()

+-------+-------+-------+
|cluster|user_id|book_id|
+-------+-------+-------+
|      0|     10|      1|
|      0|     10|      2|
|      1|     10|      3|
|      1|     10|      4|
|      1|     10|      5|
|      1|     10|      6|
|      0|      1|      1|
|      0|      1|      2|
|      1|      1|      3|
|      1|      1|      4|
|      1|      1|      5|
|      1|      1|      6|
+-------+-------+-------+

+-------+-------+-------+
|cluster|user_id|book_id|
+-------+-------+-------+
|      1|      1|      6|
|      1|      1|      3|
|      0|      1|      1|
|      0|      1|      2|
|      1|     10|      6|
|      1|      1|      5|
|      0|     10|      1|
|      1|      1|      4|
+-------+-------+-------+



In [13]:
need_cluster_df = cluster_count_df.select('user_id', 'cluster', 'old_rec',\
(F.col('count') / F.when(F.col('sum') == 0, F.lit(0.1)).otherwise(F.col('sum'))\
* (F.lit(book_number) - F.col('old_rec'))).cast(IntegerType()).alias('need'))

need_cluster_df.show()
need_cluster = need_cluster_df.collect()

+-------+-------+-------+----+
|user_id|cluster|old_rec|need|
+-------+-------+-------+----+
|     10|      1|      0|   0|
|     10|      0|      0|   0|
+-------+-------+-------+----+



In [16]:
from gorani.spark import write_api
from gorani.utils import uuid

out = []
for row in need_cluster:
    out += candidate_df.where((F.col('cluster') == row['cluster']) & (F.col('user_id') == row['user_id']))\
                .orderBy(F.rand())\
                .limit(row['need'])\
                .drop(F.col('cluster'))\
                .collect()
    if row['need'] != (book_number - row['old_rec']):
        need = book_number - row['need']
        out += candidate_df\
            .where(F.col('user_id') == row['user_id'])\
            .orderBy(F.rand())\
            .limit(need)\
            .drop(F.col('cluster'))\
            .collect()

if len(out) != 0:
    result_df = spark.createDataFrame(out)
    result_df = result_df.withColumn('id', uuid())\
        .withColumn('updated_at', F.current_timestamp())\
        .withColumn('created_at', F.current_timestamp())
    result_df.show()
    write_api('recommended_books', result_df)
    

print('success')

Py4JJavaError: An error occurred while calling o650.jdbc.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 1 in stage 248.0 failed 4 times, most recent failure: Lost task 1.3 in stage 248.0 (TID 5508, 183.99.225.238, executor 0): java.sql.BatchUpdateException: Batch entry 0 INSERT INTO recommended_books ("user_id","book_id","id","updated_at","created_at") VALUES (10,6,'d9411e66-4375-486c-ba9b-75382f9e13b4','2019-05-26 15:05:34.071+09','2019-05-26 15:05:34.071+09') was aborted: ERROR: duplicate key value violates unique constraint "recommended_books_pkey"
  Detail: Key (user_id, book_id)=(10, 6) already exists.  Call getNextException to see other errors in the batch.
	at org.postgresql.jdbc.BatchResultHandler.handleError(BatchResultHandler.java:148)
	at org.postgresql.core.ResultHandlerDelegate.handleError(ResultHandlerDelegate.java:50)
	at org.postgresql.core.v3.QueryExecutorImpl.processResults(QueryExecutorImpl.java:2184)
	at org.postgresql.core.v3.QueryExecut